# Data scraping, preparation, exploration, and pre-processing

## Data scraping

We scrape article metadata using the New York Times API. For our project we will analyze articles published between May 15th, 2023 and May 14th, 2024. We start by importing a few libraries and setting up the API.  
> **Note:** In order to run the following code you will need to create an account on (https://developer.nytimes.com) and obtain your own API key. You will then need to create a file `keys.py` in your local repo directory, and save your API key as a string under the variable name `NYT_API_KEY`.  

In [8]:
import pandas as pd
from pynytimes import NYTAPI
from datetime import datetime
from keys import NYT_API_KEY

nyt_api = NYTAPI(NYT_API_KEY, parse_dates=True)


We then fetch the data one month at a time and store in a new `.csv` file. Note that the API has a limit of 5 calls per minute (see the API [FAQ's](https://developer.nytimes.com/faq#a11)), so we need to make sure to wait until 12 seconds between consecutive calls. Note that there is also a limit of 500 calls a day.  
We wrap the process in a function so we can easily adjust the start and end date or change the path of the destination file. The function will also check whether the file already exists, and in that case give a warning before executing the procedure. This will help control the risk of restarting the relatively time-consuming process and limit unnecessary calls to the NYT servers.  
Since we are only interested in articles, we also filter out all other document types during the process.  

In [52]:
from os import listdir
from dateutil.rrule import rrule, MONTHLY
from time import sleep

def scrape(start_date: datetime, end_date: datetime, dest: str='data/nyt_metadata.csv') -> None:
    # Avoid restarting the process if the .csv already exists...
    execute = dest.split('/')[-1] not in listdir(''.join(dest.split('/')[:-1]))
    # ...unless the user confirms otherwise.
    if not execute:
        execute = input(f'A file already exists at `{dest}`. Do you want to continue?').lower() in ['y', 'yes']
    
    if execute:
        df = pd.DataFrame()
        for month in rrule(MONTHLY, dtstart=start_date, until=end_date):
            print(f'Working on the month of {month}')
            monthly_df = pd.DataFrame(nyt_api.archive_metadata(date=month))
            monthly_df = monthly_df.loc[monthly_df.pub_date.apply(lambda x: str(x)) >= str(start_date)]
            monthly_df = monthly_df.loc[monthly_df.pub_date.apply(lambda x: str(x)) < str(end_date)]
            monthly_df = monthly_df.loc[monthly_df.document_type=='article']
            if not monthly_df.empty:
                df = pd.concat([df, monthly_df]) if not df.empty else monthly_df
            sleep(12)   # The NYT servers have a limit of 5 calls per minute.
        
        df.to_csv(dest)

In [54]:
scrape(datetime(2023, 5, 15), datetime(2024, 5, 15))

Working on the month of 2023-05-15 00:00:00
Working on the month of 2023-06-15 00:00:00
Working on the month of 2023-07-15 00:00:00
Working on the month of 2023-08-15 00:00:00
Working on the month of 2023-09-15 00:00:00
Working on the month of 2023-10-15 00:00:00
Working on the month of 2023-11-15 00:00:00
Working on the month of 2023-12-15 00:00:00
Working on the month of 2024-01-15 00:00:00
Working on the month of 2024-02-15 00:00:00
Working on the month of 2024-03-15 00:00:00
Working on the month of 2024-04-15 00:00:00
Working on the month of 2024-05-15 00:00:00
